In [13]:
import pandas as pd
import numpy as np
import re #re= regular expressins to clean text like number ,lowercase ,specail symbols
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding

What is tokenizer?
it converts text to numbers
why need> neural networks cannot read text
"I love AI" ❌
[12, 45, 7] ✅




Why padding is need?
example sentences
"I love AI"
"Deep learning is very powerful"
"AI"
After tokenization, they become numbers
[4, 7, 2]                 # "I love AI"
[10, 6, 3, 8, 9]          # "Deep learning is very powerful"
[2]                       # "AI"
Lengths are 3,5,1
Neural networks require fixed-size inputs
[
  [4, 7, 2],
  [10, 6, 3, 8, 9],
  [2]
]
Because:

Arrays must be rectangular

GPU computation needs same shape
What does it do?

It adds zeros to short sequences so all sequences have the same length.
[
 [0, 0, 4, 7, 2],
 [10, 6, 3, 8, 9],
 [0, 0, 0, 0, 2]
]


Sequential is a model container in Keras that lets you build a neural network layer by layer in order
think of it as a straight pipeline
it is called sequential because .layers are added one after another
.output of one layer --> input of the next
Input → Layer1 → Layer2 → Layer3 → Output
🔷 Why Do We Use Sequential?
✅ 1. Simplicity (Very Important for Learning)

For most beginner and intermediate models:

Text classification

Image classification

Regression

RNN / LSTM models

Sequential is:

Easy to read

Easy to debug

Easy to train
✅ 2. Perfect for Linear Models

If your architecture looks like this:

Embedding → RNN → Dense


Then Sequential is the best choice.

In [14]:
data = pd.read_csv('swiggy.csv')
print("Colums in the dataset:")
print(data.columns.tolist())

Colums in the dataset:
['ID', 'Area', 'City', 'Restaurant Price', 'Avg Rating', 'Total Rating', 'Food Item', 'Food Type', 'Delivery Time', 'Review']


In [15]:
data["Review"] = data["Review"].str.lower()
data["Review"] = data["Review"].replace(r'[^a-z0-9\s]','',regex=True)

data['sentiment'] = data['Avg Rating'].apply(lambda x: 1 if x > 3.5 else 0)
data = data.dropna() #Removes all rows in the DataFrame that have missing values (NaN).
#mean rating greater than 3.5 become 1 and other 0 
#lambda x: x + 1
#mean def func(x):
#return x + 1
#What is lambda?A small anonymous function.

In [16]:
max_features = 5000
max_length = 200

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data["Review"])#fit_on_texts reads all text in the column.Builds internal mapping:Each unique word → integer.More frequent words get smaller indices
#tokenizer.word_index
# Example output:
#{'the': 1, 'and': 2, 'good': 3, 'product': 4, ...}
#Only the most frequent 5000 words are kept (because of num_words=5000)
X = pad_sequences(tokenizer.texts_to_sequences(data["Review"]), maxlen=max_length)
y = data['sentiment'].values


We will split the data into training, validation and test sets while maintaining the class distribution.

train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) : Splits data into 80% training and 20% test sets, preserving sentiment class balance
train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train) : Further splits training data into 90% training and 10% validation sets, keeping class distribution consistent

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)

In [19]:
model = Sequential([
    Embedding(input_dim=max_features, output_dim=16,
              input_length=max_length),
    SimpleRNN(64, activation='tanh', return_sequences=False),
    Dense(1, activation='sigmoid')
])

#Sequential = Keras model where layers are stacked one after another.Simple and easy to use for most feedforward and RNN/LSTM models.
#Turns integers (word indices) into vectors (embeddings)

#input_dim=max_features → size of vocabulary (top 5000 words)

#output_dim=16 → each word becomes a 16-dimensional vector

#input_length=max_length → input sequences are of length 200
#activation='tanh'

#Activation function for the hidden state

#tanh outputs values in [-1, 1]

#It allows the network to represent positive and negative signals

#Very common in RNNs
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

return_sequences	Output shape	Use case
False (default)	(batch_size, units)	Only last hidden state → good for classification
True	(batch_size, sequence_length, units)	Return hidden state at each time step → useful for sequence-to-sequence tasks like translation or text generation
Dense layer = fully connected layer
a) 1 → number of neurons

Output layer မှာ neuron တစ်ခု ရှိတယ်

Binary classification (positive/negative sentiment) မှာ 1 neuron လုံလောက်တယ်

Output = probability of positive class

b) activation='sigmoid'

Sigmoid function = 
𝜎
(
𝑥
)
=
1
1
+
𝑒
−
𝑥
σ(x)=
1+e
−x
1
	​


Input value (from RNN) → converts to range [0, 1]

Meaning: probability of positive sentiment
1️⃣ loss='binary_crossentropy'

Loss function = model error ကို measure လုပ်တာ

Binary classification (positive/negative sentiment) အတွက် အသုံးပြုတယ်

Formula:

loss
=
−
1
𝑁
∑
𝑖
=
1
𝑁
[
𝑦
𝑖
log
⁡
(
𝑝
𝑖
)
+
(
1
−
𝑦
𝑖
)
log
⁡
(
1
−
𝑝
𝑖
)
]
loss=−
N
1
	​

i=1
∑
N
	​

[y
i
	​

log(p
i
	​

)+(1−y
i
	​

)log(1−p
i
	​

)]

𝑦
𝑖
y
i
	​

 = true label (0 or 1)

𝑝
𝑖
p
i
	​

 = predicted probability (output of sigmoid)

Meaning:

Model prediction နဲ့ true label ကွာခြားမှုကို quantify လုပ်တယ်

Loss လျော့သွား → Model လေ့လာမှုကောင်းလာတာ
Metrics = training & validation performance ကို track

accuracy → percentage of correct predictions

In [20]:
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

score = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {score[1]:.2f}")

Epoch 1/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 12s 41ms/step - accuracy: 0.7149 - loss: 0.6019 - val_accuracy: 0.7156 - val_loss: 0.5999
Epoch 2/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.7160 - loss: 0.5981 - val_accuracy: 0.7156 - val_loss: 0.6026
Epoch 3/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.7160 - loss: 0.5980 - val_accuracy: 0.7156 - val_loss: 0.5962
Epoch 4/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.7160 - loss: 0.5971 - val_accuracy: 0.7156 - val_loss: 0.5958
Epoch 5/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.7160 - loss: 0.5969 - val_accuracy: 0.7156 - val_loss: 0.5969
Test accuracy: 0.72


| Parameter                        | Explanation                                                        |
| -------------------------------- | ------------------------------------------------------------------ |
| `X_train, y_train`               | Training data (inputs and labels)                                  |
| `epochs=5`                       | Dataset ကို **5 ကြိမ်** iterate လုပ်မယ် (full pass)                |
| `batch_size=32`                  | **32 sample** တစ်ခုချင်း batch လုပ်ပြီး weight update လုပ်မယ်      |
| `validation_data=(X_val, y_val)` | Training အတွင်း performance စစ်ဖို့ **validation set** ကို သုံးမယ် |
| `verbose=1`                      | Training progress ကို **output terminal** မှာ print                |
score = model.evaluate(X_test, y_test, verbose=0)
Purpose: Final test evaluation after training

Test set = completely unseen data

Returns a list: [loss, accuracy] (because metrics=['accuracy'] in compile)

In [21]:
def predict_sentiment(review_text):
    text = review_text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)

    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_length)

    prediction = model.predict(padded)[0][0]
    return f"{'Positive' if prediction >= 0.5 else 'Negative'} (Probability: {prediction:.2f})"


sample_review = "The food was great."
print(f"Review: {sample_review}")
print(f"Sentiment: {predict_sentiment(sample_review)}")

Review: The food was great.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step
Sentiment: Positive (Probability: 0.72)
